chrome://dino/

In [1]:
from __future__ import division

In [2]:
# ! pip install autopy

In [3]:
# ! pip install tensorflow

In [4]:
from autopy import *
import gc
from PIL import Image, ImageOps
import numpy as np
import time
import random
from matplotlib import cm
import os.path

In [17]:
import win32gui, win32com.client
import re

class WindowMgr:
    """Encapsulates some calls to the winapi for window management"""
    def __init__ (self):
        """Constructor"""
        self._handle = None

    def find_window(self, class_name, window_name = None):
        """find a window by its class_name"""
        self._handle = win32gui.FindWindow(class_name, window_name)

    def _window_enum_callback(self, hwnd, wildcard):
        '''Pass to win32gui.EnumWindows() to check all the opened windows'''
        if re.match(wildcard, str(win32gui.GetWindowText(hwnd))) != None:
            self._handle = hwnd

    def find_window_wildcard(self, wildcard):
        self._handle = None
        win32gui.EnumWindows(self._window_enum_callback, wildcard)
        
   
    def find_window_position_size(self):
        rect = win32gui.GetWindowRect(self._handle)
        x = rect[0]
        y = rect[1]
        w = rect[2] - x
        h = rect[3] - y
        return (x , y , w , h)
    
    def set_foreground(self):
        """put the window in the foreground"""
        shell = win32com.client.Dispatch("WScript.Shell")
        shell.SendKeys('%')
        win32gui.SetForegroundWindow(self._handle)
        
    def get_screen_image(self):
        x , y , w , h = self.find_window_position_size()
        bmp = bitmap.capture_screen()
        bmp.save('screen.png')
        img = Image.open('screen.png')
        img = img.crop((x + 10 , y + 150  , x+w -10 , y+ h - 200 ))
        img = img.resize((256,256)).convert('L')
        img = ImageOps.invert(img)
        shape = [(0, 55), (w, 50)]
        img = np.asarray(img)
        img = img / 255.0
        img = img.reshape((256,256))
        return img

In [6]:
def is_done(image):
    avg = np.average(image[52:55 , 0:255])
    if avg < 0.05:
        return True 
    else:
        return False

In [7]:
def key_press(code):
        key.toggle(code , True)
        key.toggle(code , False)

In [8]:
def key_tap(code):
    key.tap(code , delay = 0.00000001)
#     key.toggle(code , True)
#     time.sleep(0.000001)
#     key.toggle(code , False)

In [9]:
class Gym:
    def __init__(self):
        self.window = None
        
    def reset(self):
        self.window = WindowMgr()
        self.window.find_window_wildcard(".*chrome://dino/.*")
        self.window.set_foreground()
        key_press(key.Code.F5)
        time.sleep(0.08)
        key_press(key.Code.SPACE)
        img = np.zeros((256,256,3))
        return img
    
    def step(self , action , delay):
        
        
        if action == 2 :
            key_press(key.Code.UP_ARROW)
        if action == 0 :
            key_tap(key.Code.UP_ARROW)
        if action == 3 :
            key_press(key.Code.DOWN_ARROW)
        if action == 1 :
            pass
#         delay = 0 if 0.1 - delay <=0 else 0.1 - delay
#         time.sleep(delay)
        img1 = self.window.get_screen_image()
        time.sleep(1/50)
        img2 = self.window.get_screen_image()
        time.sleep(1/50)
        img3 = self.window.get_screen_image()
        
        img = np.stack([img1 , img2 , img3] , axis=-1)

        done = is_done(img3)


        if done : 
            reward =  +1
        else:
            reward = -1
  
        return img , done , reward

In [10]:
env = Gym()
# env.reset()
# done = True
# while done :
#     observation,done,reward = env.step(np.random.randint(0,3))
# #     im = Image.fromarray(np.uint8(cm.gist_earth(observation)*255))
# #     display(im)

In [11]:
from tensorflow.keras import layers
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

In [12]:
class iDQNAgent:
    def __init__(self, state_shape , action_no, memory_size = 1500):
        self.action_no = action_no
        self.memory = []
        self.memory_size = memory_size
        self.gamma = 0.9
        self.epsilon = 1.0
        self.epsilon_min = 0.1
        self.epsilon_decay = 0.999
        self.learning_rate = 1e-4
        self.sync_nets = 50

        
        self.state_shape = state_shape

        
        self.q_model = self.build_model(state_shape , action_no)
        self.q_model.compile(loss='mse' , optimizer=Adam(learning_rate=self.learning_rate))
        
#         if os.path.exists('q_model.h5'):
#             self.q_model.load_weights('q_model.h5')
            
        self.target_q_model = self.build_model(state_shape , action_no)
        
        self.update_weights()
    
        
        ####################################################
    def build_model(self ,input_shape , n_outputs):
        inputs = layers.Input(shape = input_shape , name='state')
        x = layers.Conv2D(64, (4,4) , strides=4 ,padding='same' , kernel_initializer='he_normal')(inputs)
        x = layers.Activation('relu')(x)
        x = layers.BatchNormalization()(x)
        
        x = layers.Conv2D(64, (4,4) , strides=2 ,padding='same' , kernel_initializer='he_normal')(x)
        x = layers.Activation('relu')(x)
        x = layers.BatchNormalization()(x)
        
        x = layers.Conv2D(64, (3,3) , strides=2 ,padding='same' , kernel_initializer='he_normal')(x)
        x = layers.Activation('relu')(x)
        x = layers.BatchNormalization()(x)
#         x = layers.MaxPooling2D()(x)
        
        x = layers.Conv2D(64, (3,3) , strides=2 ,padding='same' , kernel_initializer='he_normal')(x)
        x = layers.Activation('relu')(x)
        x = layers.BatchNormalization()(x)
#         x = layers.MaxPooling2D()(x)
        
        x = layers.Conv2D(64, (3,3) , strides=2 ,padding='same' , kernel_initializer='he_normal')(x)
        x = layers.Activation('relu')(x)
        x = layers.BatchNormalization()(x)
#         x = layers.MaxPooling2D()(x)
   
        x = layers.Flatten()(x) 

        x = layers.Dense(512 , activation='relu')(x)
        x = layers.Dense(256 , activation='relu')(x)
        x = layers.Dense(n_outputs , activation='linear' , name = 'action')(x)
        model = Model(inputs , x)
#         model.summary()
        return model
    
        #####################################################
    def act(self , state):
        
        if np.random.rand() < self.epsilon:
            return np.random.randint(0 , self.action_no) , 0
            
        t = time.time()
        q_values = self.q_model.predict(np.expand_dims(state , axis=0))
        action = np.argmax(q_values[0])
        print("action {} q-value {} ".format(action , q_values[0][action] ))
        return action , time.time()-t
    
        ######################################################
    def remember(self, state, action, reward , next_state , done):
        item = (state , action , reward , next_state ,done)
        if len(self.memory) > self.memory_size :
            self.memory.pop(0)
        self.memory.append(item)
        
        #######################################################
    def get_target_q_value(self, next_state, reward , double):
        if double :
            action = np.argmax(self.q_model.predict(next_state[0]))
            q_value = self.target_q_model.predict(next_state)[0][action]
        else :
            q_value = np.amax(self.target_q_model.predict(next_state)[0])
        
        q_value *= self.gamma
        q_value += reward
        return q_value
    
        ########################################################
    def replay(self , batch_size):
        sars_batch = random.sample(self.memory , batch_size)
       
        state_batch , q_value_batch = [] , []
        for state, action, reward, next_state, done in sars_batch:
            
            q_values = self.q_model.predict(np.expand_dims(state , axis=0))
            
            q_value = self.get_target_q_value(np.expand_dims(next_state , axis=0), reward, False)
            # ??????????
            q_values[0][action] = reward if done==False else q_value
            
            state_batch.append(state)
            q_value_batch.append(q_values[0])
        
#         print('q-values mean: {}'.format(np.mean(q_value_batch)))
        
        hist = self.q_model.fit(np.array(state_batch) , 
                             np.array(q_value_batch),
                             batch_size = batch_size,
                            epochs = 1,
                            verbose = 0)
                
        return hist.history['loss'][0]
       
       ######################################################

       ######################################################
    def update_epsilon(self ):
#         self.epsilon = self.epsilon_min + (1 - self.epsilon_min)* np.exp(-self.epsilon_decay * episode)
        self.epsilon = self.epsilon  * self.epsilon_decay
        if self.epsilon < self.epsilon_min :
            self.epsilon = self.epsilon_min

        ########################################################
    def update_weights(self):
            self.target_q_model.set_weights(self.q_model.get_weights())


In [19]:
episode_count = 1000
time_steps =    100
batch_size = 16

In [14]:
agent = iDQNAgent((256 , 256 , 3) ,4)

In [21]:
all_rewards = []
all_losses =  []
loss = 0

for episode in range(episode_count):
    state = env.reset()
    done = True
    total_reward = 0

    for t in range(time_steps) :
        action , delay = agent.act(state)
        next_state , done ,reward = env.step(action , delay)
#         if not done:
#             reward -= 100
        agent.remember(state , action , reward , next_state , done)
        state = next_state
        total_reward += reward
   
        if not done : 
            break
            
    if len(agent.memory) >= batch_size:
        loss = agent.replay(batch_size)
        all_losses.append(loss) 
        
    agent.update_epsilon()
    
    if episode % agent.sync_nets == 0:
        agent.update_weights()
        print('update weights')
        print("Avarage loss {}".format(np.mean(np.array(all_losses[-50:]))) )
        print("Avarage reward {}".format(np.mean(np.array(all_rewards[-50:]))) )
        
    if episode % 100 == 0:
        agent.q_model.save_weights('q_model.h5')
        
    print("episode {} reward {} loss {} epsilon {} memory {}".format(episode , total_reward, loss, agent.epsilon , len(agent.memory)))
    all_rewards.append(total_reward)
    
    if loss > 2 :
        break

1/1 [==============================] - 0s 22ms/step
update weights
Avarage loss 0.8525865077972412
Avarage reward nan
episode 0 reward 100 loss 0.8525865077972412 epsilon 0.999 memory 100


C:\Users\nguye\anaconda3\envs\maicg\lib\site-packages\numpy\core\fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\nguye\anaconda3\envs\maicg\lib\site-packages\numpy\core\_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


1/1 [==============================] - 0s 21ms/step
episode 1 reward 100 loss 0.574608564376831 epsilon 0.998001 memory 200
1/1 [==============================] - 0s 23ms/step
episode 2 reward 100 loss 0.4906129837036133 epsilon 0.997002999 memory 300
1/1 [==============================] - 0s 23ms/step
episode 3 reward 100 loss 0.6238020658493042 epsilon 0.996005996001 memory 400
1/1 [==============================] - 0s 24ms/step
episode 4 reward 100 loss 0.40300267934799194 epsilon 0.995009990004999 memory 500
1/1 [==============================] - 0s 23ms/step
episode 5 reward 100 loss 0.4987751841545105 epsilon 0.994014980014994 memory 600
1/1 [==============================] - 0s 24ms/step
episode 6 reward 100 loss 0.5962080955505371 epsilon 0.993020965034979 memory 700
1/1 [==============================] - 0s 20ms/step
action 3 q-value 0.010112438350915909 
1/1 [==============================] - 0s 21ms/step
action 3 q-value 0.04031495004892349 
1/1 [============================

KeyboardInterrupt: 

In [25]:
model = agent.q_model.load_weights('q_model.h5')
state = env.reset()
done = True
while done :
    q_values = agent.q_model.predict(np.expand_dims(state , axis=0))
    action = np.argmax(q_values[0])
    next_state , done ,reward  = env.step(action ,0)
    state = next_state
#     im = Image.fromarray(np.uint8(cm.gist_earth(observation)*255))
#     display(im)

1/1 [==============================] - 0s 26ms/step
